In [ ]:
import numpy as np
import pandas as pd
from tqdm import trange
from gymnasium.utils.env_checker import check_env
from flygym.mujoco import Parameters
from flygym.mujoco.examples.turning_controller import HybridTurningNMF
from flygym.mujoco.examples.turning_controller import CPGNetwork
from flygym.mujoco.arena import FlatTerrain

# Random walk function

In [ ]:
# Function define to generate descending drives used for the random walk
def generate_random_walk(num_steps):
    # Initialise the firts turning coefficient
    turnings = [np.array([0,0])]
    # Initialize the step counting variable
    count_step_turn = 0
    # INitialize stabilizinf step counter
    total_step_turn = 100

    # Iterate over all the given number of step
    for i in range(num_steps - 1):

        # Firts stabilizing step drives equal to 0
        if count_step_turn <= total_step_turn:
            sigma_l = turnings[-1][0]
            sigma_r = turnings[-1][1]
        
        # For all the next steps
        else:
            count_step_turn = 0

            # Randomly choose the left descending drives among four different normal probabilities and their probalities
            while True:
                sigma_l = np.random.choice([np.random.normal(1, 0.3),
                                                      np.random.normal(1.25, 0.3),
                                                      np.random.normal(0.40, 0.3),
                                                      np.random.normal(-1.25, 0.3)],
                                                      p=[0.2, 0.3, 0.2, 0.3])
                # absolute values superior to 1.5 are not allowed (not relevant)
                if sigma_l >= -1.3 and sigma_l <= 1.3 :
                    break

            # Update normal distribution probabilities based on the previously selected left coefficient
            if sigma_l > 1.0:
                proba_r = np.array([0.1, 0.1, 0.5, 0.3]) # increase right turn probability
            elif (sigma_l > 0.2 and sigma_l < 0.4):
                proba_r = np.array([0.05, 0.9, 0.05, 0.0]) # increase straight trajectory or smooth turn probability
            elif (sigma_l > -1.0 and sigma_l <= 0.2):
                proba_r = np.array([0.05, 0.9, 0.05, 0.0]) # increase straight trajectory or smooth turn probability
            elif sigma_l < -1.0:
                proba_r = np.array([0.1, 0.3, 0.5, 0.1]) # increase left turn probability
            else:
                proba_r = np.array([0.4, 0.2, 0.2, 0.2]) # increase straight trajectory or smooth turn probability

            # Choose the right coefficient dependent on the new normal distribution probabilities
            while True:
                sigma_r = np.random.choice([np.random.normal(1, 0.3),
                                                      np.random.normal(1.25, 0.2),
                                                      np.random.normal(0.40, 0.2),
                                                      -sigma_l],
                                                      p = proba_r)
                # absolute values superior to 1.5 are not allowed (not relevant)
                if sigma_r >= -1.2 and sigma_l <= 1.2 :
                    break

            # If large difference between both coefficient (= large turn) counting step variable has higher probabilities to be larger
            if np.abs(sigma_l - sigma_r) > 0.65:
                total_step_turn = np.random.gamma(140, 65)
            else:
                total_step_turn = np.random.gamma(50, 30)
         
        # Increment the step counter variable
        count_step_turn += 1

        # Stor each coefficient combination at each time step
        turnings.append(np.array([sigma_l,sigma_r]))
        
    return turnings

# Initialize simulation

In [ ]:
# Define important variables for the simulation
run_time = 1
timestep = 1e-4
arena = FlatTerrain((100,100))

contact_sensor_placements = [
    f"{leg}{segment}"
    for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]

# Define simulation parameters
sim_params = Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_camera="Animat/camera_top",
    render_playspeed=0.2,
    enable_adhesion=True,
    draw_adhesion=False,
    actuator_kp=20,
)

# We use the pre-programmed Turning controller
nmf = HybridTurningNMF(
    sim_params=sim_params,
    arena = arena,
    spawn_pos=(0, 0, 0.2),
    contact_sensor_placements=contact_sensor_placements,
)

# Check environment
check_env(nmf)

### Simulation and CSV storage

In [ ]:
# Define the number of deisred simulations
n_sim = 1

# Iterate over all separate simulations
for n in range (n_sim):

    # Vector to store fetaures history
    speed_integration = np.array([0.0,0.0])
    #angle = 100
    drive_hist = []
    fly_position_hist = []
    heading_hist = []
    speed_integration_hist = []
    velocity_hist = []
    swing_hist = []
    contact_forces_hist_LF = []
    contact_forces_hist_LM = []
    contact_forces_hist_LH = []
    contact_forces_hist_RF = []
    contact_forces_hist_RM = []
    contact_forces_hist_RH = []
    position_hist_LF = []
    position_hist_LM = []
    position_hist_LH = []
    position_hist_RF = []
    position_hist_RM = []
    position_hist_RH = []

    #turns = []

    # Reset the NMF class
    obs, info = nmf.reset()

    # Define spawn position
    nmf.spawn_pos=(0, 0, 0.2),

    # Define a fresh CPG network at the beginning of each simulation
    nmf.cpg_network = CPGNetwork(
                    timestep= nmf.sim_params.timestep,
                    intrinsic_freqs=nmf.intrinsic_freqs,
                    intrinsic_amps=nmf.intrinsic_amps,
                    coupling_weights=nmf.coupling_weights,
                    phase_biases=nmf.phase_biases,
                    convergence_coefs=nmf.convergence_coefs,
                    seed=0,
                )
    
    # Generate random walk for given number of timesteps in two separted variables
    turns_1 = generate_random_walk(39500)
    turns_2 = generate_random_walk(39500)

    # Iterate over each steps using HybridTurning controller
    for i in trange(80000):

        # Stabilizing first steps
        if i < 1000:
            action = np.array([0,0])
            obs, reward, terminated, truncated, info = nmf.step(action)

            # Path integration using speed vector
            speed_integration[0] += obs["fly"][1,0]
            speed_integration[1] += obs["fly"][1,1]  

        # First turning coefficients
        elif (i >= 1000) and (i < len(turns_1)+1000):
            # Action equal to the pre-defined descending drives coefficients
            action = turns_1[i-1000]
            obs, reward, terminated, truncated, info = nmf.step(action)

            # Path integration using speed vector
            speed_integration[0] += obs["fly"][1,0]
            speed_integration[1] += obs["fly"][1,1]

        elif i >= len(turns_1)+1000:
            # Action equal to the pre-defined descending drives coefficients
            action = turns_2[i % len(turns_2)]
            obs, reward, terminated, truncated, info = nmf.step(action)

            # Path integration using speed vector
            speed_integration[0] += obs["fly"][1,0]
            speed_integration[1] += obs["fly"][1,1]
        
        obs, reward, terminated, truncated, info = nmf.step(action)
        nmf.render()

        # Swing index defined in preprogrammed steps
        swing_index = nmf.preprogrammed_steps.swing_period
        # CPG phase
        cpg_phase = nmf.cpg_network.curr_phases
        cpg_phase = cpg_phase % np.pi
        # Swing Stance history
        swing_hist.append([swing_index[key][0] <= value <= swing_index[key][1] for key, value in zip(swing_index.keys(), cpg_phase)])
        # Store all interesting features in history lists
        drive_hist.append(action)
        fly_position_hist.append(obs["fly"][0])
        heading_hist.append(obs["fly_orientation"])
        speed_integration_hist.append(speed_integration)
        velocity_hist.append(obs["fly"][1,:])
        contact_forces_hist_LF.append(obs["contact_forces"][5])
        contact_forces_hist_LM.append(obs["contact_forces"][11])
        contact_forces_hist_LH.append(obs["contact_forces"][17])
        contact_forces_hist_RF.append(obs["contact_forces"][23])
        contact_forces_hist_RM.append(obs["contact_forces"][29])
        contact_forces_hist_RH.append(obs["contact_forces"][35])
        position_hist_LF.append(obs["end_effectors"][0])
        position_hist_LM.append(obs["end_effectors"][1])
        position_hist_LH.append(obs["end_effectors"][2])
        position_hist_RF.append(obs["end_effectors"][3])
        position_hist_RM.append(obs["end_effectors"][4])
        position_hist_RH.append(obs["end_effectors"][5])


    # Save a video for each simulation
    nmf.save_video(f"./outputs/loop_test_{n+1}.mp4")

    # Save all features into a dataframe
    df = pd.DataFrame({
        'action_drive': drive_hist,
        'fly_position': fly_position_hist,
        'heading': heading_hist,
        'speed_integration': speed_integration_hist,
        'velocity': velocity_hist,
        'swing_phase': swing_hist,
        'contact_forces_hist_LF': contact_forces_hist_LF,
        'contact_forces_hist_LM': contact_forces_hist_LM,
        'contact_forces_hist_LH': contact_forces_hist_LH,
        'contact_forces_hist_RF': contact_forces_hist_RF,
        'contact_forces_hist_RM': contact_forces_hist_RM,
        'contact_forces_hist_RH': contact_forces_hist_RH,
        'position_hist_LF': position_hist_LF,
        'position_hist_LM': position_hist_LM,
        'position_hist_LH': position_hist_LH,
        'position_hist_RF': position_hist_RF,
        'position_hist_RM': position_hist_RM,
        'position_hist_RH': position_hist_RH})
    
    # Save the dataframe into a csv file
    df.to_csv('./outputs/csv_random_walk.csv', mode='a', index=None, header= (n==0))